In [1]:
Pkg.add( "CUDAdrv")
Pkg.add("CUDAnative")
Pkg.add("CuArrays")

UndefVarError: UndefVarError: Pkg not defined

In [2]:
#Pkg.add("StatsBase")

In [3]:
using StatsBase: wsample


In [4]:
cd(@__DIR__)

In [5]:
#Pkg.add("Flux")

In [6]:
#Pkg.test("Flux")

In [7]:
using Flux

In [8]:
shakespear = open("100-0.txt") do file
    read(file, String)
end
print("B")

B

In [9]:
train = shakespear[3000:end]
length(train)

5793398

In [10]:
text = collect(String(train))
length(text)

5793398

In [11]:
alphabet = [unique(text)..., '_']
length(alphabet)

101

In [12]:
using Flux:onehot
text = map(ch -> onehot(ch, alphabet), text)

5793398-element Array{Flux.OneHotVector,1}:
 [true, false, false, false, false, false, false, false, false, false  …  false, false, false, false, false, false, false, false, false, false]
 [false, true, false, false, false, false, false, false, false, false  …  false, false, false, false, false, false, false, false, false, false]
 [true, false, false, false, false, false, false, false, false, false  …  false, false, false, false, false, false, false, false, false, false]
 [false, false, true, false, false, false, false, false, false, false  …  false, false, false, false, false, false, false, false, false, false]
 [false, false, true, false, false, false, false, false, false, false  …  false, false, false, false, false, false, false, false, false, false]
 [false, false, true, false, false, false, false, false, false, false  …  false, false, false, false, false, false, false, false, false, false]
 [false, false, true, false, false, false, false, false, false, false  …  false, false, fals

In [13]:
stop = onehot('_', alphabet)

101-element Flux.OneHotVector:
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
     ⋮
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false

In [14]:
N = length(alphabet)
seqlen = 50
nbatch = 50


50

In [15]:
N

101

In [16]:
using Flux: batchseq, chunk
using Base.Iterators: partition
Xs = collect(partition(batchseq(chunk(text,nbatch), stop), seqlen))
Ys = collect(partition(batchseq(chunk(text[2:end], nbatch),stop), seqlen))


2318-element Array{Array{Flux.OneHotMatrix{Array{Flux.OneHotVector,1}},1},1}:
 [[false true … false false; true false … false false; … ; false false … false false; false false … false false], [true false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false true; … ; false false … false false; false false … false false], [false false … false true; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false 

In [17]:
m = Chain(
    LSTM(N,128),
    LSTM(128,128),
#     LSTM(128,128),
    Dense(128,N),
    softmax)

m = gpu(m)

Chain(Recur(LSTMCell(101, 128)), Recur(LSTMCell(128, 128)), Dense(128, 101), NNlib.softmax)

In [18]:
using Flux: throttle, crossentropy

function loss(xs, ys)
    l = sum(crossentropy.(m.(gpu.(xs)), gpu.(ys)))
    Flux.truncate!(m)
    return l
end

loss (generic function with 1 method)

In [19]:
?Flux.Optimise.ADAM

```
ADAM(params, η = 0.001; β1 = 0.9, β2 = 0.999, ϵ = 1e-08, decay = 0)
```

[ADAM](https://arxiv.org/abs/1412.6980v8) optimiser.


In [20]:
opt = Flux.Optimise.ADAM(params((m),0.01))

#43 (generic function with 1 method)

In [21]:
?Flux.train!

```
train!(loss, data, opt)
```

For each datapoint `d` in `data` computes the gradient of `loss(d...)` through backpropagation and calls the optimizer `opt`.

Takes a callback as keyword argument `cb`. For example, this will print "training" every 10 seconds:

```julia
Flux.train!(loss, data, opt,
            cb = throttle(() -> println("training"), 10))
```

The callback can return `:stop` to interrupt the training loop.

Multiple optimisers and callbacks can be passed to `opt` and `cb` as arrays.


In [ ]:
using Flux.Tracker
tx, ty = (Xs[5],Ys[5])
evalcb = () -> @show loss(tx,ty)
data = zip(Xs, Ys)
Flux.train!(loss, data, opt, cb = throttle(evalcb,5))

loss(tx, ty) = 229.61530791771045 (tracked)
loss(tx, ty) = 182.86205824924852 (tracked)
loss(tx, ty) = 168.29415203919885 (tracked)
loss(tx, ty) = 165.5025523638618 (tracked)
loss(tx, ty) = 164.1891773515537 (tracked)
loss(tx, ty) = 164.04224135727688 (tracked)
loss(tx, ty) = 163.9116877030349 (tracked)
loss(tx, ty) = 163.86663577861935 (tracked)
loss(tx, ty) = 163.73232309371005 (tracked)
loss(tx, ty) = 163.58731165844898 (tracked)
loss(tx, ty) = 163.52490970141451 (tracked)
loss(tx, ty) = 163.43092951165266 (tracked)
loss(tx, ty) = 163.23384136293492 (tracked)


In [ ]:
function sample(m, alphabet, len; temp = 1)
  m = gpu(m)
  Flux.reset!(m)
  buf = IOBuffer()
  c = rand(alphabet)
  for i = 1:len
    write(buf, c)
    c = wsample(alphabet, m(onehot(c, alphabet)).data)
  end
  return String(take!(buf))
end

sample(m, alphabet, 1000) |> println
